In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import random 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
'''
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras.layers import MaxPooling2D

In [ ]:
TRAINDATADIR = "../input/intel-image-classification/seg_train/seg_train"
TESTDATADIR = "../input/intel-image-classification/seg_test/seg_test"
CATEGORIES = ['buildings','forest','glacier','mountain','sea','street']

In [ ]:
IMG_SIZE = 150

In [ ]:
import os

In [ ]:
def create_data(DATADIR, CATEGORIES):
    data = []
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
            res_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            data.append([res_array, class_num])
    return data

In [ ]:
def one_hot_encode (label):
    label = np_utils.to_categorical(label)
    return label

In [ ]:
train_data = create_data(TRAINDATADIR, CATEGORIES)
print(len(train_data))

test_data = create_data(TESTDATADIR, CATEGORIES)
print(len(test_data))

In [ ]:
random.shuffle(train_data)
random.shuffle(test_data)

In [ ]:
train_images = []
label_train = []
for sample in train_data:
    train_images.append(np.array(sample[0]))
    label_train.append(sample[1])

test_images = []
label_test = []
for sample in test_data:
    test_images.append(np.array(sample[0]))
    label_test.append(sample[1])

In [ ]:
train_images = np.asarray(train_images)
test_images = np.asarray(test_images)

In [ ]:
labels_train = one_hot_encode(label_train)
labels_test = one_hot_encode(label_test) 

In [ ]:
print(train_images.shape)
print(labels_train.shape)
print(test_images.shape)
print(labels_test.shape)

## Splitting

In [ ]:
def splitColor (dataset, rows, columns):
    
    split_datasets = [ [] for i in range( rows * columns ) ]
    
    rows = int (150 / rows)
    columns = int (150 / columns)
    
    for data in dataset:
        data = data.reshape( 150//rows, rows,-1, columns, 3).swapaxes(1,2).reshape(-1, rows, columns, 3)
        
        for i in range(len(split_datasets)):
            split_datasets[i].append(data[i])
    
    return np.array(split_datasets)

In [ ]:
def display(image, rows, columns):

    fig = plt.figure(figsize = (5,5))

    for i in range( rows * columns):

        fig.add_subplot(rows, columns, i + 1)

        plt.imshow(image[i], cmap = "gray")
        plt.axis('off')
    plt.show()

In [ ]:
split_datasets_train4 = splitColor(train_images, 2, 2)
split_datasets_test4 = splitColor(test_images, 2, 2)

In [ ]:
split_datasets_train9 = splitColor(train_images, 3, 3)
split_datasets_test9 = splitColor(test_images, 3, 3)

In [ ]:
split_datasets_train6 = splitColor(train_images, 3, 2)
split_datasets_test6 = splitColor(test_images, 3, 2)

In [ ]:
print(split_datasets_train9.shape)

## Model

In [ ]:
def combine(predictions):
    p = []
    for j in range(len(predictions[0])):
        y = np.array([np.argmax(predictions[i][j]) for i in range(len(predictions))])
        p.append(np.bincount(y).argmax())
    return np_utils.to_categorical(p)

## VGG16

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
# load model without classifier layers
model = VGG16(include_top=False, input_shape=(32, 32, 3))

for layer in model.layers:
        layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(11, activation='softmax')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_images, labels_train,validation_data=(test_images, labels_test), epochs=10, batch_size = 128,verbose=2)

In [ ]:
net4 = []
# load model without classifier layers
for i in range(4):
    net4.append(VGG16(include_top=False, input_shape=(48, 48, 3)))

for i in range(4):
    for layer in net4[i].layers:
        layer.trainable = False

for i in range(4):
    # add new classifier layers
    flat1 = Flatten()(net4[i].layers[-1].output)
    class1 = Dense(512, activation='relu')(flat1)
    output = Dense(11, activation='softmax')(class1)
    # define new model
    net4[i] = Model(inputs=net4[i].inputs, outputs=output)

net4[i].summary()

In [ ]:
for i in range(4):
    net4[i].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    net4[i].fit(split_datasets_train4[i], labels_train, epochs=10, batch_size = 128,verbose=2)

In [ ]:
net9 = []
# load model without classifier layers
for i in range(9):
    net9.append(VGG16(include_top=False, input_shape=(50, 50, 3)))

for i in range(9):
    for layer in net9[i].layers:
        layer.trainable = False
        
for i in range(9):
    # add new classifier layers
    flat1 = Flatten()(net9[i].layers[-1].output)
    class1 = Dense(512, activation='relu')(flat1)
    output = Dense(6, activation='softmax')(class1)
    # define new model
    net9[i] = Model(inputs=net9[i].inputs, outputs=output)
    
net9[0].summary()

In [ ]:
for i in range(9):
    net9[i].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    net9[i].fit(split_datasets_train9[i], labels_train, epochs=10, batch_size = 128,verbose=2)

In [ ]:
net6 = []
# load model without classifier layers
for i in range(6):
    net6.append(VGG16(include_top=False, input_shape=(32, 32, 3)))

for i in range(6):
    for layer in net6[i].layers:
        layer.trainable = False
        
for i in range(6):
    # add new classifier layers
    flat1 = Flatten()(net6[i].layers[-1].output)
    class1 = Dense(512, activation='relu')(flat1)
    output = Dense(11, activation='softmax')(class1)
    # define new model
    net6[i] = Model(inputs=net6[i].inputs, outputs=output)
    
net6[0].summary()

In [ ]:
for i in range(6):
    net6[i].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    net6[i].fit(split_datasets_train6[i], labels_train, epochs=10, batch_size = 128,verbose=2)

## ResNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten

In [ ]:
Rnet9 = []
# load model without classifier layers
for i in range(9):
    Rnet9.append(ResNet50(include_top=False, input_shape=(50, 50, 3)))

for i in range(9):
    for layer in Rnet9[i].layers:
        layer.trainable = False

for i in range(9):
    # add new classifier layers
    flat1 = Flatten()(Rnet9[i].layers[-1].output)
    class1 = Dense(512, activation='relu')(flat1)
    output = Dense(6, activation='softmax')(class1)
    # define new model
    Rnet9[i] = Model(inputs=Rnet9[i].inputs, outputs=output)
    # summarize
Rnet9[0].summary()

In [ ]:
for i in range(9):
    Rnet9[i].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    Rnet9[i].fit(split_datasets_train9[i], labels_train, epochs=10, batch_size = 128,verbose=2)

## Predcitions

### VGG16

In [ ]:
vgg16pred4 = [ [] for i in range(len(net4))]

for i in range(len(net4)):
    vgg16pred4[i] = net4[i].predict(split_datasets_test4[i])

In [ ]:
vgg16pred9 = [ [] for i in range(len(net9))]

for i in range(len(net9)):
    vgg16pred9[i] = net9[i].predict(split_datasets_test9[i])

In [ ]:
vgg16pred6 = [ [] for i in range(len(net6))]

for i in range(len(net6)):
    vgg16pred6[i] = net6[i].predict(split_datasets_test6[i])

### ResNet50

In [ ]:
ResNet50pred4 = [ [] for i in range(len(net4))]

for i in range(len(net4)):
    ResNet50pred4[i] = net4[i].predict(split_datasets_test4[i])

In [ ]:
ResNet50pred9 = [ [] for i in range(len(net9))]

for i in range(len(net9)):
    ResNet50pred9[i] = Rnet9[i].predict(split_datasets_test9[i])

In [ ]:
ResNet50pred6 = [ [] for i in range(len(net6))]

for i in range(len(net6)):
    ResNet50pred6[i] = Rnet6[i].predict(split_datasets_test6[i])

## Combine

### VGG16

In [ ]:
vgg16p4 = combine(vgg16pred4)

In [ ]:
vgg16p9 = combine(vgg16pred9)

In [ ]:
vgg16p6 = combine(vgg16pred6)

### ResNet50

In [ ]:
ResNet50p4 = combine(ResNet50pred4)

In [ ]:
ResNet50p9 = combine(ResNet50pred9)

In [ ]:
ResNet50p6 = combine(ResNet50pred6)

## Accuracy

### VGG16

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,vgg16p4)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,vgg16p9)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,vgg16p6)
metric.result().numpy()

### ResNet50

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,ResNet50p4)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,ResNet50p9)
metric.result().numpy()

In [ ]:
from keras.metrics import Accuracy
metric = Accuracy()
metric.update_state(labels_test,ResNet50p6)
metric.result().numpy()

# Single

### VGG16

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
# load model without classifier layers
model = VGG16(include_top=False, input_shape=(75, 75, 3))

for layer in model.layers:
        layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(6, activation='softmax')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(data_train, labels_train,validation_data=(data_test, labels_test), epochs=10, batch_size = 128,verbose=2)

### ResNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
# load model
model = ResNet50()
# summarize the model
input_tensor = Input(shape=(75, 75, 3))

model = ResNet50(input_tensor=input_tensor,include_top=False)

for layer in model.layers:
        layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(6, activation='softmax')(class1)
# define new model
model = Model(inputs=model.inputs, outputs=output)
# summarize
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(data_train, labels_train,validation_data=(data_test, labels_test), epochs=10, batch_size = 128,verbose=2)